# Symbolic nonlinear observability

In [ ]:
import numpy as np
import sympy as sp
from IPython.display import display

In [ ]:
# Import functions directly from github
# Important: note that we use raw.githubusercontent.com, not github.com

import requests
url = 'https://raw.githubusercontent.com/florisvb/Nonlinear_and_Data_Driven_Estimation/main/Utility/symbolic_derivatives.py'
r = requests.get(url)

# Store the file to the colab working directory
with open('symbolic_derivatives.py', 'w') as f:
    f.write(r.text)

# import the function we want from that file
import symbolic_derivatives

# Example: downward facing constant altitude monocular camera

Here we have a single camera pointed down, moving laterally at constant altitude.

$
\mathbf{\dot{x}} = \mathbf{f}(\mathbf{x},\mathbf{u}) =
\frac{d}{dt}
\begin{bmatrix}
\bbox[yellow]{g} \\[0.3em]
\bbox[yellow]{z} \\[0.3em]
\end{bmatrix} =
\overset{f_0}{\begin{bmatrix}
0 \\[0.3em]
0
\end{bmatrix}} +
\overset{f_1}{\begin{bmatrix}
1 \\[0.3em]
0
\end{bmatrix}} \bbox[lightgreen]{u}
$

We have 1 measurement, the ventral optic flow. We can also assume we have lateral acceleration measurements, but in this case the acceleration is entirely defined by the control inputs we don't have to add it explicitly.

$
\mathbf{y} = \mathbf{{h}}(\mathbf{{x}}, \mathbf{{u}}) =
\begin{bmatrix}
\bbox[yellow]{g/z} \\[0.3em]
\end{bmatrix}
$

# Define states and dynamics in control affine form

In [ ]:
g, z = sp.symbols(['g', 'z'])
x = [g, z]

f_0 = sp.Matrix([0,
                 0])
f_1 = sp.Matrix([1,
                 0])

# Define measurements

In [ ]:
h = sp.Matrix([g/z])

# Calculate each term in G

$G = [h, L_{f_0}h, L_{f1}h]$

In [ ]:
# Take the derivative of h with respect to x along the vector f_0
L_f0_h = symbolic_derivatives.directional_derivative(h, x, f_0)
display(L_f0_h)

print('')

# Take the derivative of h with respect to x along the vector f_1
L_f1_h = symbolic_derivatives.directional_derivative(h, x, f_1)
display(L_f1_h)

Matrix([[0]])

Matrix([[1/z]])

# Assemble G, take Jacobian

In [ ]:
G = sp.Matrix([h, L_f0_h, L_f1_h])
display(G)

Matrix([
[g/z],
[  0],
[1/z]])

In [ ]:
G.jacobian(x)

Matrix([
[1/z, -g/z**2],
[  0,       0],
[  0, -1/z**2]])

# Check the rank of G for a given operating point, $x_0$

In [ ]:
x0 = {g: 1, z: 2}
display(G.jacobian(x).subs(x0))

print('')
print('Rank of G:')
G.jacobian(x).subs(x0).rank()

Matrix([
[1/2, -1/4],
[  0,    0],
[  0, -1/4]])


Rank of G:


2

# Shortcut function to get G:

### First derivatives

In [ ]:
# First derivatives
G1 =  symbolic_derivatives.get_bigO(h, x,[f_0, f_1])

# Second derivatives
G2 =  symbolic_derivatives.get_bigO(sp.Matrix.vstack(*G1), x, [f_0, f_1] )

# Both first and second derivatives
G = sp.Matrix.vstack(*G1, *G2)
display(G)

Matrix([
[g/z],
[  0],
[1/z],
[g/z],
[  0],
[1/z],
[  0],
[  0],
[  0],
[1/z],
[  0],
[  0]])

# Exercises:

1. Is the system observable with no controls (i.e. $u=0$)?
2. Is the system observable with control? (i.e. $u\neq0$)?
3. How many derivatives are needed, 1 or 2?
3. Apply the symbolic approach to the planar drone example with the measurements below. What is necessary in order for $z$ to be observable?